# Предобработка телеметрии самосвалов (продолжение)
Добавление дополнительной информации из разных таблиц

          ДВС.xlsx
        (Марка ДВС, MTBF...)
              ^
              | (по модели двигателя)
              |
Масло (oil_all) ----(truck_number, время asof)----> Телеметрия
   ^                                                     ^
   | (truck_number)                                      | (mdm_object_id = OBJECTID)
   +-------------------------- idles.csv ----------------+




In [1]:
import os
import tqdm
import warnings

import pandas as pd
from pandas.api.types import is_numeric_dtype

### Функция оптимизации памяти

In [2]:
def optimize_dtypes(df):
    df_optimized = df.copy()

    # float64 → float32
    float_cols = df_optimized.select_dtypes(include='float64').columns
    df_optimized[float_cols] = df_optimized[float_cols].astype('float32')

    # int64 → int32
    int_cols = df_optimized.select_dtypes(include='int64').columns
    df_optimized[int_cols] = df_optimized[int_cols].astype('int32')

    # object → category (для строк, где мало уникальных значений)
    object_cols = df_optimized.select_dtypes(include='object').columns
    for col in object_cols:
        num_unique_values = df_optimized[col].nunique()
        num_total_values = len(df_optimized[col])
        if num_unique_values / num_total_values < 0.5:
            df_optimized[col] = df_optimized[col].astype('category')

    return df_optimized


def setup_pandas_options():
    """Настройка глобальных опций pandas для отображения."""
    pd.set_option("display.precision", 3)
    pd.set_option("expand_frame_repr", False)


setup_pandas_options()

source_root = '../dataset'
files = os.listdir(source_root)
print(files)

['.DS_Store', 'telemetry_filtered_filled.csv', 'ДВС.xlsx', 'Масляная лаборатория 1.xlsx', 'zip', 'РейсыБарж.xlsx', 'idles.csv', 'telemetry_filtered_filled_2.csv', 'oil.csv']


In [6]:
# Посмотрим содержание имеющихся файлов
for file in files:
    filepath = os.path.join(source_root, file)

    # Пропускаем системные файлы
    if file.startswith('.'):
        continue

    print(f"\n=== {file} ===")
    if file.lower().endswith('.csv'):
        try:
            df = pd.read_csv(filepath)
            display(df.head())
        except Exception as e:
            print(f"Ошибка чтения CSV: {e}")
    elif file.lower().endswith('.xlsx'):
        try:
            df = pd.read_excel(filepath)
            display(df.head())
        except Exception as e:
            print(f"Ошибка чтения XLSX: {e}")


=== telemetry_filtered_filled.csv ===


,create_dt,mdm_object_id,mdm_object_name,alt,speed_gps,inst_fuel,weight,temp_engine,turn_engine,load_engine,...,torque_nn,pres_des_rail_injector_nn,dynamic_brake,mdm_object_uuid,meta_object_name,mdm_model_name,dfm_in_hour,dfm_out_sum,dfm_out_hour,meta_model_name
0,2024-01-01 12:23:42+11:00,1661,1395,-66.0,0.0,293.0,19.0,66.0,6425.0,4.0,...,126.0,94.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins
1,2024-01-01 12:23:47+11:00,1661,1395,-66.0,0.0,257.0,34.0,66.0,6395.0,5.0,...,127.0,99.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins
2,2024-01-01 12:23:52+11:00,1661,1395,-66.0,0.0,263.0,35.0,66.0,6409.0,1.0,...,127.0,98.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins
3,2024-01-01 12:23:57+11:00,1661,1395,-66.0,0.0,494.0,35.0,66.0,6392.0,4.0,...,127.0,101.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins
4,2024-01-01 12:24:02+11:00,1661,1395,-66.0,0.0,431.0,34.0,66.0,6387.0,3.0,...,127.0,88.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins



=== ДВС.xlsx ===


,Марка ДВС,Ремонты за 2023-24 год,Unnamed: 2,Unnamed: 3,Средняя наработка на ремонт за 2023г,Количество в ремонте на 12.03.2024г,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Количество,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,АТС,КАМСС,ММФИ
3,КТТА-19,3,NaN,NaN,12-18 тмч,1,0,0
4,КТА-50,10,NaN,NaN,21-25 тмч,1,2,0



=== Масляная лаборатория 1.xlsx ===


...

,IDLEID,OBJECTID,OBJECT_EXPORT_CODE,GMTBEGINTIME,GMTENDTIME,EXPORT_CODE,IDLETYPEID,PLACE,DELETED,IS_MANUAL,...,EXPLOITATION_START,EXPLOITATION_END,IDLETYPEID.1,IDLETYPENAME,CATEGORYID,DIVISIONID,OBJECTTYPEID.1,ISREPAIR,DECREASES_KTG,DECREASES_KI
0,34957434,1381,NaN,2023-01-04 13:01:00.000,2023-01-04 13:05:16.000,NaN,70.0,NaN,0,0,...,2019-04-12 15:00:00.000,NaN,70.0,Погрузка,83.0,1.0,1.0,0.0,0.0,0.0
1,34957434,1381,NaN,2023-01-04 13:01:00.000,2023-01-04 13:05:16.000,NaN,70.0,NaN,0,0,...,2019-04-12 15:00:00.000,NaN,70.0,Погрузка,83.0,1.0,161.0,0.0,0.0,0.0
2,34957473,1581,NaN,2023-01-04 13:02:54.000,2023-01-04 13:06:13.000,NaN,70.0,NaN,0,0,...,2019-05-18 15:00:00.000,NaN,70.0,Погрузка,83.0,1.0,1.0,0.0,0.0,0.0
3,34957473,1581,NaN,2023-01-04 13:02:54.000,2023-01-04 13:06:13.000,NaN,70.0,NaN,0,0,...,2019-05-18 15:00:00.000,NaN,70.0,Погрузка,83.0,1.0,161.0,0.0,0.0,0.0
4,34957550,1383,NaN,2023-01-04 13:07:31.000,2023-01-04 13:10:29.000,NaN,70.0,NaN,0,0,...,2019-05-04 15:00:00.000,NaN,70.0,Погрузка,83.0,1.0,1.0,0.0,0.0,0.0



=== telemetry_filtered_filled_2.csv ===


,create_dt,mdm_object_id,mdm_object_name,alt,speed_gps,inst_fuel,weight,temp_engine,turn_engine,load_engine,...,pres_des_rail_injector_nn,dynamic_brake,mdm_object_uuid,meta_object_name,mdm_model_name,dfm_in_hour,dfm_out_sum,dfm_out_hour,meta_model_name,is_stopped
0,2024-01-01 12:23:42+11:00,1661,1395,-66.0,0.0,293.0,19.0,66.0,6425.0,4.0,...,94.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins,1
1,2024-01-01 12:23:47+11:00,1661,1395,-66.0,0.0,257.0,34.0,66.0,6395.0,5.0,...,99.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins,1
2,2024-01-01 12:23:52+11:00,1661,1395,-66.0,0.0,263.0,35.0,66.0,6409.0,1.0,...,98.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins,1
3,2024-01-01 12:23:57+11:00,1661,1395,-66.0,0.0,494.0,35.0,66.0,6392.0,4.0,...,101.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins,1
4,2024-01-01 12:24:02+11:00,1661,1395,-66.0,0.0,431.0,34.0,66.0,6387.0,3.0,...,88.0,0.0,83397e13-90c4-11ec-98b9-00155d5fc801,1395,БелАЗ 75306 Cummins,0.0,0.0,0.0,БелАЗ 75306 Cummins,1



=== oil.csv ===


,SampleId,ComponentRef,UnitRef,CustomerId,ReportedDate,TakenDate,ReceivedDate,ViscMode,Particle,Parker,...,FleetIdField,ComponentDescField,ComponentTypeField,LocationField,ComponentEngineSizeField,TurboField,GearTypeField,ComponentModelField,ComponentMakeField,ComponentIdField
0,44442,ML200011DVS68,ML200011BEL215988,ML200011OOO953,2023-04-02 11:57:35.000,2023-04-01 00:00:00.000,2023-04-01 00:00:00.000,Measured,False,True,...,0,CP-00001263,Diesel Engine,CENTER,0,True,NaN,QSK60,CUMMINS,DVS 1374
1,44443,ML200011GS 462,ML200011BEL215988,ML200011OOO953,2023-04-02 12:24:35.000,2023-04-01 00:00:00.000,2023-04-01 00:00:00.000,Measured,True,True,...,0,CP-00001264,Hydraulic,CENTER,0,False,NaN,75306,BELAZ,GS 1374
2,44457,ML200011GS 378,ML200011BEL250,ML200011OOO953,2023-04-02 17:58:15.000,2023-04-02 17:57:47.000,2023-04-02 17:57:47.000,Measured,True,True,...,0,CP-00001275,Hydraulic,CENTER,0,False,NaN,75306,BELAZ,GS 1381
3,44458,ML200011DVS541,ML200011BEL250,ML200011OOO953,2023-04-02 18:21:22.000,2023-04-02 18:20:43.000,2023-04-02 18:20:43.000,Measured,False,True,...,0,CP-00001278,Diesel Engine,CENTER,0,True,NaN,75306,BELAZ,DVS 1381
4,46632,ML200011QSK77,ML200011BEL745,ML200011OOO953,2023-04-14 11:51:42.000,2023-04-13 00:00:00.000,2023-04-13 00:00:00.000,Measured,False,True,...,0,CP-00001474,Diesel Engine,CENTER,0,True,NaN,75306,BELAZ,QSK60 1349


## 2. Загрузка и объединение данных

In [7]:
telemetry = pd.read_csv(os.path.join(source_root, 'telemetry_filtered_filled_2.csv'))
idles = pd.read_csv(os.path.join(source_root, 'idles.csv'))
oil_lab1 = pd.read_excel(os.path.join(source_root, 'Масляная лаборатория 1.xlsx'))
oil_lab2 = pd.read_csv(os.path.join(source_root, 'oil.csv'))
dvs = pd.read_excel(os.path.join(source_root, 'ДВС.xlsx'))

print('telemetry:', telemetry.shape)
print('idles:', idles.shape)
print('oil_lab1:', oil_lab1.shape)
print('oil_lab2:', oil_lab2.shape)
print('dvs:', dvs.shape)

telemetry: (43751198, 24)
idles: (585830, 35)
oil_lab1: (13827, 39)
oil_lab2: (147, 39)
dvs: (7, 8)


In [8]:
## 3. Объединяем таблицы масла
common_cols = sorted(set(oil_lab1.columns) & set(oil_lab2.columns))

oil_all = pd.concat(
    [oil_lab1[common_cols], oil_lab2[common_cols]],
    ignore_index=True
)

print('oil_all (объединённое масло):', oil_all.shape)

oil_all (объединённое масло): (13974, 39)


In [9]:
# 4. Преобразование дат
# Телеметрия
telemetry['create_dt'] = pd.to_datetime(telemetry['create_dt'])
# Масло — используем TakenDate, если есть, иначе ReportedDate
time_col = 'TakenDate' if 'TakenDate' in oil_all.columns else 'ReportedDate'
oil_all['sample_time'] = pd.to_datetime(
    oil_all[time_col].fillna(oil_all.get('ReportedDate'))
)
# Простои
idles['GMTBEGINTIME'] = pd.to_datetime(idles['GMTBEGINTIME'])
idles['GMTENDTIME'] = pd.to_datetime(idles['GMTENDTIME'])

### Как сделать надёжный ключ truck_number:
1. Если UnitNumberField числовой — используем его.
2. Если там строки — пытаемся вытащить из них число.
3. Если с числом не получилось — берём последнюю часть из CustUnitIdField.
4. Храним truck_number всегда как строку (и в масле, и в телеметрии, и в простоях).
---
#### Коротко логика
- Если UnitNumberField — нормальные числа: ОК.
- Если там строки ('A954', "100", "1374", NaN и пр.):
  - to_numeric(..., errors='coerce') превратит всё нечисловое в NaN;
  - для NaN мы берём номер из CustUnitIdField (в твоих примерах он красивый: BELAZ 75306 1374 → "1374").
- Везде приводим к str, чтобы не ловить неожиданности при merge.

In [11]:
# 5. Создание общего ключа truck_number

# ---------- МАСЛО: truck_number ----------
if 'UnitNumberField' in oil_all.columns:
    unit_raw = oil_all['UnitNumberField']

    if is_numeric_dtype(unit_raw):
        # Всё хорошо, числа: 100.0, 1374.0 и т.п.
        oil_all['truck_number'] = unit_raw.astype('Int64').astype(str)
    else:
        # Смешанные / строковые значения: пробуем вытащить числа
        # 1) попытка привести к числам
        unit_num = pd.to_numeric(unit_raw, errors='coerce')

        # 2) где получилось — используем как номер борта
        truck_number_from_unit = unit_num.astype('Int64').astype(str)

        # 3) fallback: берём номер из CustUnitIdField (последнее слово)
        if 'CustUnitIdField' in oil_all.columns:
            fallback_from_cust = (
                oil_all['CustUnitIdField']
                .astype(str)
                .str.split()
                .str[-1]          # "BELAZ 75306 1374" -> "1374"
            )
        else:
            fallback_from_cust = pd.Series(index=oil_all.index, dtype='object')

        # 4) если unit_num не получилось (NaN/<NA>), подставляем fallback
        oil_all['truck_number'] = truck_number_from_unit.mask(
            truck_number_from_unit.isin(['<NA>', 'nan', 'NaN']),
            fallback_from_cust
        )

else:
    # Вообще нет UnitNumberField — сразу из CustUnitIdField
    oil_all['truck_number'] = (
        oil_all['CustUnitIdField']
        .astype(str)
        .str.split()
        .str[-1]
    )

# На всякий случай: truck_number как строка
oil_all['truck_number'] = oil_all['truck_number'].astype(str)

# ---------- ТЕЛЕМЕТРИЯ: truck_number ----------
telemetry['truck_number'] = telemetry['mdm_object_name'].astype(str)

# ---------- ПРОСТОИ: truck_number ----------
idles['truck_number'] = idles['OBJECTNAME'].astype(str)


In [13]:
# 6. Оптимизация типов (после дат и ключей!)
telemetry = optimize_dtypes(telemetry)
idles = optimize_dtypes(idles)
oil_all = optimize_dtypes(oil_all)
dvs = optimize_dtypes(dvs)

print(telemetry.info(memory_usage='deep'))
print(idles.info(memory_usage='deep'))
print(oil_all.info(memory_usage='deep'))
print(dvs.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43751198 entries, 0 to 43751197
Data columns (total 25 columns):
 #   Column                     Dtype                    
---  ------                     -----                    
 0   create_dt                  datetime64[ns, UTC+11:00]
 1   mdm_object_id              int32                    
 2   mdm_object_name            int32                    
 3   alt                        float32                  
 4   speed_gps                  float32                  
 5   inst_fuel                  float32                  
 6   weight                     float32                  
 7   temp_engine                float32                  
 8   turn_engine                float32                  
 9   load_engine                float32                  
 10  pres_coolant_nn            float32                  
 11  pres_rail_injector_nn      float32                  
 12  pres_temp_engine_nn        float32                  
 13  torque_nn 

In [20]:
# === 1. Приводим время к одному виду: datetime64[ns] без tz ===

# Телеметрия
telemetry['create_dt'] = pd.to_datetime(telemetry['create_dt'], utc=True)
telemetry['create_dt'] = telemetry['create_dt'].dt.tz_convert(None)

# Масло
time_col = 'TakenDate' if 'TakenDate' in oil_all.columns else 'ReportedDate'
oil_all['sample_time'] = pd.to_datetime(
    oil_all[time_col].fillna(oil_all.get('ReportedDate')),
    utc=True
)
oil_all['sample_time'] = oil_all['sample_time'].dt.tz_convert(None)

# === 2. Ключи по машине — точно строки ===

telemetry['truck_number'] = telemetry['truck_number'].astype(str)
oil_all['truck_number']   = oil_all['truck_number'].astype(str)

# === 3. Убираем строки без времени (NaT) ===

telemetry_no_na = telemetry.dropna(subset=['create_dt'])
oil_no_na       = oil_all.dropna(subset=['sample_time'])

# === 4. Подмножество столбцов телеметрии ===

telemetry_cols_for_merge = [
    'truck_number',
    'create_dt',
    'mdm_object_id',
    'meta_model_name',
    'temp_engine',
    'turn_engine',
    'load_engine',
    'inst_fuel',
    'speed_gps',
]
telemetry_cols_for_merge = [c for c in telemetry_cols_for_merge
                            if c in telemetry_no_na.columns]

# === 5. merge_asof ПО КАЖДОМУ БОРТУ ОТДЕЛЬНО ===

merged_chunks = []

# группируем масло по борту
for truck, oil_grp in oil_no_na.groupby('truck_number'):
    # телеметрия для этого борта
    tel_grp = telemetry_no_na[telemetry_no_na['truck_number'] == truck]

    if tel_grp.empty:
        # если для этого борта нет телеметрии — просто добавляем масло как есть
        merged_chunks.append(oil_grp.copy())
        continue

    # сортировка по времени (внутри одного борта теперь точно будет ОК)
    oil_grp_sorted = oil_grp.sort_values('sample_time')
    tel_grp_sorted = tel_grp.sort_values('create_dt')[telemetry_cols_for_merge]

    merged = pd.merge_asof(
        oil_grp_sorted,
        tel_grp_sorted,
        left_on='sample_time',
        right_on='create_dt',
        direction='backward',
        allow_exact_matches=True,
    )

    merged_chunks.append(merged)

# Финальный датасет: масло + телеметрия
oil_telemetry = pd.concat(merged_chunks, ignore_index=True)

print('oil_telemetry (масло + телеметрия):', oil_telemetry.shape)


oil_telemetry (масло + телеметрия): (13974, 51)


Предиктивная модель строится вокруг события контроля.
Поэтому правильно мержить именно масло → телеметрия, а не наоборот. Тк телеметрия — это time series с миллионами точек, а масло — это события раз в N часов.

Телеметрия:
- 43 751 198 строк
- поток сигналов раз в секунду/5 секунд

Масло:
- 13 974 строк
- раз в 250–500 моточасов

Если попытаться записать «масло в телеметрию», то получится:
 - либо 43 млн строк, где почти все значения масла NaN
 - либо неработоспособный датасет, где одно событие растянуто на сотни тысяч записей

Это полностью разрушит ML-постановку задачи.

Целевая переменная (Y) — это состояние узла, которое известно только в моменты анализа масла

Например:
- Condition = Normal / Abnormal / Severe
- или EvaluationComment = Acceptable / Unacceptable / Critical

То есть Y существует только в моменты масла.

А телеметрия должна быть агрегирована относительно момента пробы масла, например:
- средняя температура двигателя за последние 24 часа
- максимальная температура за последние 8 часов
- средняя нагрузка за последние 10 часов
- количество событий перегрева за последние 7 дней

Именно поэтому каждая строка в итоговом датасете должна соответствовать одной пробе масла.
И телеметрию тогда можно будет использовать как признаки (X).

In [23]:
display(oil_telemetry.head())

,AnalysisMode,ComponentDescField,ComponentEngineSizeField,ComponentIdField,ComponentMakeField,ComponentModelField,ComponentRef,ComponentTypeField,Condition,CustUnitDescField,...,truck_number_x,truck_number_y,create_dt,mdm_object_id,meta_model_name,temp_engine,turn_engine,load_engine,inst_fuel,speed_gps
0,Off-Road,NaN,0,DEIDVUD 180L,BUKSIR,"""PRIBOY""",ML200011DEI647,GearBox,Abnormal,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Off-Road,NaN,0,ACG350,BUKSIR,"""PRIBOY""",ML200011ACG774,GearBox,Abnormal,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Off-Road,RAID-8,0,REDUKTOR 20082939,CUMMINS,ZF350,ML200011RED643,GearBox,Normal,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Off-Road,RAID-8,0,43203820 N14-M,CUMMINS,N14-M,ML200011432593,Diesel Engine,Abnormal,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Off-Road,RAID-8,0,HRP REDUKTOR,CUMMINS,HRP 5111,ML200011HRP773,GearBox,Abnormal,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# 8. Масло + статистика по двигателям (ДВС.xlsx)
# Допустим, в масле модель двигателя в столбце 'ComponentModelField'
engine_model_col = 'ComponentModelField'
if engine_model_col in oil_telemetry.columns:
    oil_telemetry['engine_model'] = oil_telemetry[engine_model_col].astype(str)
else:
    # если нет, можно оставить пустым или подобрать другое поле
    oil_telemetry['engine_model'] = pd.NA

# В ДВС.xlsx модель двигателя в 'Марка ДВС'
dvs_renamed = dvs.rename(columns={'Марка ДВС': 'engine_model'})

oil_teletry_dvs = oil_telemetry.merge(
    dvs_renamed,
    on='engine_model',
    how='left',
    suffixes=('', '_dvs')
)

print('oil_full (масло + телеметрия + ДВС):', oil_teletry_dvs.shape)
oil_teletry_dvs.to_csv(os.path.join(source_root, 'equipment.csv'), index=False)
display(oil_teletry_dvs.head())

oil_full (масло + телеметрия + ДВС): (13974, 59)


,AnalysisMode,ComponentDescField,ComponentEngineSizeField,ComponentIdField,ComponentMakeField,ComponentModelField,ComponentRef,ComponentTypeField,Condition,CustUnitDescField,...,inst_fuel,speed_gps,engine_model,Ремонты за 2023-24 год,Unnamed: 2,Unnamed: 3,Средняя наработка на ремонт за 2023г,Количество в ремонте на 12.03.2024г,Unnamed: 6,Unnamed: 7
0,Off-Road,NaN,0,DEIDVUD 180L,BUKSIR,"""PRIBOY""",ML200011DEI647,GearBox,Abnormal,NaN,...,NaN,NaN,"""PRIBOY""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Off-Road,NaN,0,ACG350,BUKSIR,"""PRIBOY""",ML200011ACG774,GearBox,Abnormal,NaN,...,NaN,NaN,"""PRIBOY""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Off-Road,RAID-8,0,REDUKTOR 20082939,CUMMINS,ZF350,ML200011RED643,GearBox,Normal,NaN,...,NaN,NaN,ZF350,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Off-Road,RAID-8,0,43203820 N14-M,CUMMINS,N14-M,ML200011432593,Diesel Engine,Abnormal,NaN,...,NaN,NaN,N14-M,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Off-Road,RAID-8,0,HRP REDUKTOR,CUMMINS,HRP 5111,ML200011HRP773,GearBox,Abnormal,NaN,...,NaN,NaN,HRP 5111,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 9. (Опционально) Базовый справочник техники из телеметрии + простоев
equip_from_tel = telemetry[['mdm_object_id', 'mdm_object_name', 'meta_model_name', 'truck_number']].drop_duplicates()
equip_from_idle = idles[['OBJECTID', 'OBJECTNAME', 'MODELNAME', 'truck_number']].drop_duplicates()

equipment = equip_from_tel.merge(
    equip_from_idle,
    left_on='truck_number',
    right_on='truck_number',
    how='outer',
    suffixes=('_tel', '_idle')
)

print('equipment (справочник техники):', equipment.shape)
equipment.to_csv(os.path.join(source_root, 'equipment.csv'), index=False)
display(equipment)

equipment (справочник техники): (12, 7)


,mdm_object_id,mdm_object_name,meta_model_name,truck_number,OBJECTID,OBJECTNAME,MODELNAME
0,1381,1349,БелАЗ 75306 Cummins,1349,1381,1349,БелАЗ 75306 Б
1,1381,1349,БелАЗ 75306 Б GalileoSky,1349,1381,1349,БелАЗ 75306 Б
2,1383,1374,БелАЗ 75306 Cummins,1374,1383,1374,БелАЗ 75306 Б
3,1383,1374,БелАЗ 75306 Б GalileoSky,1374,1383,1374,БелАЗ 75306 Б
4,1581,1381,БелАЗ 75306 Cummins,1381,1581,1381,БелАЗ 75306 Б
5,1581,1381,БелАЗ 75306 Б GalileoSky,1381,1581,1381,БелАЗ 75306 Б
6,1384,1385,БелАЗ 75306 Cummins,1385,1384,1385,БелАЗ 75306 Б
7,1384,1385,БелАЗ 75306 Б GalileoSky,1385,1384,1385,БелАЗ 75306 Б
8,1661,1395,БелАЗ 75306 Cummins,1395,1661,1395,БелАЗ 75306 Б
9,1661,1395,БелАЗ 75306 Б GalileoSky,1395,1661,1395,БелАЗ 75306 Б
